<a href="https://colab.research.google.com/github/profteachkids/CHE3023/blob/main/CoolingTower.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import numpy as np
from scipy.integrate import solve_ivp
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default='plotly_dark'
from scipy.optimize import newton, root


In [46]:
h = 10
R = 8.314
Cpair = 1e3
rhoa = 1.2
Cpwv = 2000
Cpw = 4190
kc = h/(rhoa*Cpair)
Hv = 40700 #J/mol

a = 5 
Ma = 29e-3 
Mw = 18e-3

Lw = 0.5
vel = 5 
P=101325

def Pv(T):
  return (10**(8.07131 - 1730.63/(233.426+(T-273.15))))/760*101325

TaFeed = 25 + 273.15
yFeed = Pv(TaFeed)*0.5/P

TaExit_guess = TaFeed+5.
yExit_guess = yFeed*1.1

TwFeed = 70 + 273.15
Ga = vel * P*Ma/(R*TaFeed)
zend = 20
z=np.linspace(0, zend, 50)
sol=None

In [47]:
def converge(v1):
  TaExit = v1[0]
  yExit = v1[1]  
  global sol
  def dv(z,v2):
    y=v2[0]
    Tw=v2[1]
    Ta=v2[2]

    Gw = y/(1-y)*Mw/Ma * Ga
    G = Ga + Gw
    Cpa = (Cpair*Ga +  Cpwv*Gw)/G
  
    M = (Gw + Ga)/(Gw/Mw + Ga/Ma) 
    Cwi = Pv(Tw)/R/Tw
    Nw = kc*(Cwi - y*P/(R*Ta))
    dydz = -a*M/G * (Nw)
    dTwdz = a/Lw/Cpw*(h*(Ta-Tw)-Hv*Nw)
    dTadz = a/G/Cpa*(h*(Ta-Tw))

    return dydz, dTwdz, dTadz
  res = solve_ivp(dv, (0,zend), (yExit, TwFeed, TaExit),
                  method='Radau', dense_output=True)
  sol = res.sol(z)
  return sol[0,-1]-yFeed, sol[2:,-1]-TaFeed

In [48]:
converge((TaExit_guess, yExit_guess))

(-0.008765546681691048, array([3.49819343]))

In [49]:
root(converge, (TaExit_guess,yExit_guess))

/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray



    fjac: array([[-5.21746123e-05,  9.99999999e-01],
       [-9.99999999e-01, -5.21746123e-05]])
     fun: array([ 5.69400065e-11, -2.34540494e-08])
 message: 'The solution converged.'
    nfev: 6
     qtf: array([ 1.93761651e-06, -1.29963014e-08])
       r: array([  1.15727805, -28.5165548 ,  -1.10754762])
  status: 1
 success: True
       x: array([3.00319111e+02, 2.48966282e-02])

In [50]:
fig = make_subplots(rows=3, cols=1)
fig.add_scatter(x=z, y=sol[1],name='H2O Temperature',row=1,col=1)
fig.add_scatter(x=z, y=sol[2],name='Air Temperature', row=2,col=1)
fig.add_scatter(x=z, y=sol[0]*P,name='H2O Partial Pressure',row=3,col=1)
fig.update_layout(width=600, height=900)
fig.show()